### 1. Project Background

This project aims to explore and apply feature Engineering and dimensionality reduction techniques on second-hand car sales datasets. The goal is to prepare the dataset for machine learning models by creating informative features and reducing the feature space while retaining essential information.
The second-hand car market is vast, and predicting the price of a used car is a challenging task due to various factors influencing the price. Feature engineering and dimensionality reduction play a crucial role in improving the performance of predictive models by enhancing the quality of input data and reducing noise.

### 2. Load Data & Libraries

In [22]:
# Impoting general libraries
import warnings
import itertools
import random

# Data Analysis Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as mn

# Data Preparation, Modeling, and Evaluation (SKLearn Modules)
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler,RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from collections import Counter
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# Machine Learning Models 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

# Machine Learning Model Evaluation Methods 
from sklearn.metrics import mean_squared_error

warnings.filterwarnings(action="ignore")
%matplotlib inline

In [23]:
#load data
data=pd.read_csv(r'C:\Users\Admin\Desktop\Zindua_School\Unsupervised_Machine_Learning\Week1_Principal_Component_Analysis\datasets\second_hand_cars.csv')
data.head()

,Company Name,Car Name,Variant,Fuel Type,Tyre Condition,Make Year,Owner Type,Registration Number,Mileage,Price,Transmission Type,Body Color,Service Record,Insurance,Registration Certificate,Accessories
0,Maruti Suzuki,Cruze,EX,CNG,Needs Replacement,2018,Second,84-436-5584,52798,759107,Manual,Grey,Major Service at 50418 km,No Current Insurance,Not Available,"Music System, Sunroof, Alloy Wheels"
1,Kia,Seltos,RXE,Petrol,New,2020,Third,79-114-3166,43412,505071,Automatic,Maroon,Major Service at 131313 km,No Current Insurance,Available,NaN
2,Kia,Accord,RXE,Petrol,New,2022,Second,41-358-3344,95219,635322,Automatic (Tiptronic),Black,No Service Record,No Current Insurance,Available,NaN
3,Nissan,Seltos,Highline,Diesel,Used,2024,Third,92-708-1763,70370,483152,Automatic (Tiptronic),Maroon,Major Service at 98115 km,Valid Until [date],Available,"Music System, Alloy Wheels"
4,Chevrolet,Kwid,Highline,Petrol,Used,2018,Second,76-154-5485,85852,712961,Automatic (Tiptronic),Silver,Major Service at 135665 km,No Current Insurance,Not Available,"GPS, Music System"


### 3. Data Preprocessing
#### 3.1 Data Cleaning: Handling missing data


In [24]:
#check for missingness
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Company Name              2500 non-null   object
 1   Car Name                  2500 non-null   object
 2   Variant                   2238 non-null   object
 3   Fuel Type                 2500 non-null   object
 4   Tyre Condition            2500 non-null   object
 5   Make Year                 2500 non-null   int64 
 6   Owner Type                2500 non-null   object
 7   Registration Number       2500 non-null   object
 8   Mileage                   2500 non-null   int64 
 9   Price                     2500 non-null   int64 
 10  Transmission Type         2500 non-null   object
 11  Body Color                2500 non-null   object
 12  Service Record            2500 non-null   object
 13  Insurance                 2500 non-null   object
 14  Registration Certificate

Looks like only 2 features have missingness: Variant and  Accessories
Let us explore the missingness even more...

In [25]:
#how much missingness does each have
total_missing=data.isnull().sum().sort_values(ascending=False)
portion_missing=total_missing/data.isnull().count().sort_values(ascending=False)
missing_df=pd.concat([total_missing,portion_missing],axis=1,keys=['total missing','percent missing'],sort=False)
missing_df.index.name='variable'# set the index name to variable 
missing_df=missing_df[total_missing>0] # only retain those with missing values 
missing_df['percent missing']=missing_df['percent missing'].map(lambda x:x*100) # convert to percentage 
missing_df

,total missing,percent missing
variable,,
Accessories,482,19.28
Variant,262,10.48


Only a small percent of data in each feature is missing, now we can proceed to impute the data, first explore the data type- if it is categorical data impute with the mode, if it is numerical data, plot the distribution and impute either by mean or median

In [26]:
#data.Variant

In [27]:
#data.Accessories

In [28]:
data['Accessories'].fillna(data['Accessories'].mode()[0], inplace=True)
data['Variant'].fillna(data['Variant'].mode()[0], inplace=True)
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Company Name              2500 non-null   object
 1   Car Name                  2500 non-null   object
 2   Variant                   2500 non-null   object
 3   Fuel Type                 2500 non-null   object
 4   Tyre Condition            2500 non-null   object
 5   Make Year                 2500 non-null   int64 
 6   Owner Type                2500 non-null   object
 7   Registration Number       2500 non-null   object
 8   Mileage                   2500 non-null   int64 
 9   Price                     2500 non-null   int64 
 10  Transmission Type         2500 non-null   object
 11  Body Color                2500 non-null   object
 12  Service Record            2500 non-null   object
 13  Insurance                 2500 non-null   object
 14  Registration Certificate

Now we look for duplicates in the data

In [29]:
duplicated=data[data.duplicated()]
duplicated

,Company Name,Car Name,Variant,Fuel Type,Tyre Condition,Make Year,Owner Type,Registration Number,Mileage,Price,Transmission Type,Body Color,Service Record,Insurance,Registration Certificate,Accessories


There are no deplicated rows in the data frame.

### 4. Feature Engineering

#### 4.1 Creating new features

In [30]:
#Calculating the age of the car from the year of manufacturer
#first convert the column to date time data type
from datetime import datetime
data['Make Year']=pd.to_datetime(data['Make Year'])
#set the year
current_year=2024

def car_age(yom):
    current_year=datetime.today().year
    age= current_year-yom.year
    return age

In [31]:
data['Car Age']=data['Make Year'].apply(car_age)
##data.head()


In [32]:
data['Price per mile']=round(data['Price']/data['Mileage'])
data.head()

,Company Name,Car Name,Variant,Fuel Type,Tyre Condition,Make Year,Owner Type,Registration Number,Mileage,Price,Transmission Type,Body Color,Service Record,Insurance,Registration Certificate,Accessories,Car Age,Price per mile
0,Maruti Suzuki,Cruze,EX,CNG,Needs Replacement,1970-01-01 00:00:00.000002018,Second,84-436-5584,52798,759107,Manual,Grey,Major Service at 50418 km,No Current Insurance,Not Available,"Music System, Sunroof, Alloy Wheels",54,14.0
1,Kia,Seltos,RXE,Petrol,New,1970-01-01 00:00:00.000002020,Third,79-114-3166,43412,505071,Automatic,Maroon,Major Service at 131313 km,No Current Insurance,Available,Sunroof,54,12.0
2,Kia,Accord,RXE,Petrol,New,1970-01-01 00:00:00.000002022,Second,41-358-3344,95219,635322,Automatic (Tiptronic),Black,No Service Record,No Current Insurance,Available,Sunroof,54,7.0
3,Nissan,Seltos,Highline,Diesel,Used,1970-01-01 00:00:00.000002024,Third,92-708-1763,70370,483152,Automatic (Tiptronic),Maroon,Major Service at 98115 km,Valid Until [date],Available,"Music System, Alloy Wheels",54,7.0
4,Chevrolet,Kwid,Highline,Petrol,Used,1970-01-01 00:00:00.000002018,Second,76-154-5485,85852,712961,Automatic (Tiptronic),Silver,Major Service at 135665 km,No Current Insurance,Not Available,"GPS, Music System",54,8.0


#### 4.2 Data Encoding
Encode all categorical data

In [33]:
#split data into, categorical, numerical and temporal
num_df=data.select_dtypes(include=[np.number])
cat_df=data.select_dtypes(include=[object])

In [34]:
#num_df.head()
#cat_df.head()

Drop the feature registration number, because it is sort of an identifier, model learns nothing from it.

In [35]:
cat_df.drop('Registration Number',axis=1 ,inplace=True)

In [36]:
cat_df.head()

,Company Name,Car Name,Variant,Fuel Type,Tyre Condition,Owner Type,Transmission Type,Body Color,Service Record,Insurance,Registration Certificate,Accessories
0,Maruti Suzuki,Cruze,EX,CNG,Needs Replacement,Second,Manual,Grey,Major Service at 50418 km,No Current Insurance,Not Available,"Music System, Sunroof, Alloy Wheels"
1,Kia,Seltos,RXE,Petrol,New,Third,Automatic,Maroon,Major Service at 131313 km,No Current Insurance,Available,Sunroof
2,Kia,Accord,RXE,Petrol,New,Second,Automatic (Tiptronic),Black,No Service Record,No Current Insurance,Available,Sunroof
3,Nissan,Seltos,Highline,Diesel,Used,Third,Automatic (Tiptronic),Maroon,Major Service at 98115 km,Valid Until [date],Available,"Music System, Alloy Wheels"
4,Chevrolet,Kwid,Highline,Petrol,Used,Second,Automatic (Tiptronic),Silver,Major Service at 135665 km,No Current Insurance,Not Available,"GPS, Music System"


In [38]:
cat_df.columns

Index(['Company Name', 'Car Name', 'Variant', 'Fuel Type', 'Tyre Condition',
       'Owner Type', 'Transmission Type', 'Body Color', 'Service Record',
       'Insurance', 'Registration Certificate', 'Accessories'],
      dtype='object')

In [41]:
#one hot encoding

ohe=OneHotEncoder(sparse_output=False)
ohe_transform=ohe.fit_transform(cat_df)
encoded_df = pd.DataFrame(ohe_transform, columns=ohe.get_feature_names_out(cat_df.columns))
encoded_df.index = cat_df.index
final_df=pd.concat([num_df,encoded_df],axis=1)
final_df.head()

,Mileage,Price,Car Age,Price per mile,Company Name_Chevrolet,Company Name_Ford,Company Name_Honda,Company Name_Hyundai,Company Name_Kia,Company Name_Maruti Suzuki,...,"Accessories_Sunroof, Leather Seats, Music System, GPS","Accessories_Sunroof, Music System","Accessories_Sunroof, Music System, Alloy Wheels","Accessories_Sunroof, Music System, Alloy Wheels, GPS","Accessories_Sunroof, Music System, Alloy Wheels, Leather Seats","Accessories_Sunroof, Music System, GPS","Accessories_Sunroof, Music System, GPS, Alloy Wheels","Accessories_Sunroof, Music System, GPS, Leather Seats","Accessories_Sunroof, Music System, Leather Seats","Accessories_Sunroof, Music System, Leather Seats, GPS"
0,52798,759107,54,14.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,43412,505071,54,12.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,95219,635322,54,7.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,70370,483152,54,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,85852,712961,54,8.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### 4.3 Feature Scaling

In [44]:
#perform standard scaling
scaler=StandardScaler()
scaled_df=scaler.fit_transform(final_df)


### 5. Dimensionality Reduction

#### 5.1 Correlation Analysis